In [ ]:
import numpy as np
import pandas as pd
from miles_edit import *

travel = pd.read_csv('output_2023_09_05/reports/travel-4-combined.csv')
segments = pd.read_csv('output_2023_09_05/reports/overall-5-ways.csv')
intersections = pd.read_csv('output_2023_09_05/reports/overall-6-intersections.csv')
reach = pd.read_csv('output_2023_09_05/reports/overall-2-reach-type.csv')

In [ ]:
## Don't remember why I did this but interesting to look at later maybe
reach[reach["Type"] =="project total"].sort_values(by="Intersections count").reset_index().plot(y=["Intersections count",'Segments count'])

### Projects to test manually:
~~More connected/less crossings
64ff64a08cf244c982529b2f
65454061b5bfdcb11540ec4f
More long/linear with many crossings
65284187772d22a2108ddc4c
64921e2f1930d10600997fd9
NVM these don't work, probably inputted after Matt sent this data. I only have up to 64e...~~


long/linear

64921e2f1930d10600997fd9 ### ERROR ### No volume even though intersections are selected???? 5/7 + 4/6 + 5/7 + 4/7 + 5/7 + 5/7 = 28/41

64b0406741e08c5dff327a1f 79/106 selected

connected

645582371c8b985d1be43a07 19/41

64dfb34bcb0d64389a5ea11e 14/28

In [ ]:
# (This is the wrong version where I didn't convert the project distance from feet -> miles)
#Pull data and input into the function from the volume to miles code
def miles(project_id,mode, volume ,adjacent_ratio): # **NOTE: Changed the labels in volume_to_miles.json to be "bicycling" and "walking" for consistency so I only have to use one variable :P
    proj_reach = reach[reach["Project ID"] == project_id][reach["Type"] == "project total"]
    # volume = travel.loc[travel["Project ID"] == project_id][travel["M Mode"] == mode][travel["K Estimate"] == "mean"]["Existing Travel"]
    # print("The total distance travelled is: "+ str(distance)+ " miles")
    # print(proj_reach.iloc[0,4],proj_reach.iloc[0,4]/5280)
    return(calculate_distance(mode, proj_reach.iloc[0,4], proj_reach.iloc[0,2], volume, adjacent_ratio))
miles("645582371c8b985d1be43a07","bicycling",10,-1)

In [ ]:
def miles_units(project_id,mode, volume ,adjacent_ratio): # **NOTE: Changed the labels in volume_to_miles.json to be "bicycling" and "walking" for consistency so I only have to use one variable :P
    proj_reach = reach[reach["Project ID"] == project_id][reach["Type"] == "project total"]
    # volume = travel.loc[travel["Project ID"] == project_id][travel["M Mode"] == mode][travel["K Estimate"] == "mean"]["Existing Travel"]
    # print("The total distance travelled is: "+ str(distance)+ " miles")
    print(proj_reach.iloc[0,4],proj_reach.iloc[0,4]/5280)
    if mode == "bicycling":
        units = proj_reach.iloc[0,3]
    else:
        units = proj_reach.iloc[0,2]
    return(calculate_distance(mode, (proj_reach.iloc[0,4]/5280), units, volume, adjacent_ratio))

In [ ]:
miles_units("645582371c8b985d1be43a07","bicycling",200,-1)

In [ ]:
miles_units("645582371c8b985d1be43a07","walking",200,-1)

In [ ]:
miles_units("645582371c8b985d1be43a07","bicycling",200,7/14)

In [ ]:
miles_units("645582371c8b985d1be43a07","walking",200,7/14)

In [ ]:
# Test by running for all projects and compare to the existing weighted miles - do the bike numbers really look too low?
# (without the adjacent ways adjustment because I don't have that data
def apply_weighting(row):
    return (miles(row["Project ID"],row["M Mode"],row["Existing Travel"],-1))
def apply_weighting_units(row):
    return (miles_units(row["Project ID"],row["M Mode"],row["Existing Travel"],-1))

#def apply_people(row):
    #return (miles(row["Project ID"],row["M Mode"],row["Existing Travel"],0))
#def apply_people_adj(row):
    #return (miles(row["Project ID"],row["M Mode"],row["Existing Travel"], 0)["people_adj"])

travel["New Weighted Existing Travel"] = travel.apply(apply_weighting, axis=1)
travel["New Weighted Existing Travel Miles"] = travel.apply(apply_weighting_units, axis=1)

#travel["Unique People"] = travel.apply(apply_people, axis=1)
#travel["New Weighted Existing Travel"] = travel.apply(apply_people_adj, axis=1)


#for index, element in travel.iterrows():
    #travel.at[index,"New Weighted Travel"] = miles(element["Project ID"],element["M Mode"],0)
    #print(travel.loc[index,"New Weighted Travel"])

In [ ]:
travel.groupby("M Mode").plot(y=["Weighted Existing Travel","New Weighted Existing Travel","Existing Travel","New Weighted Existing Travel Miles"])

In [ ]:
travel.groupby("M Mode").plot(y=["Weighted Existing Travel","Existing Travel","New Weighted Existing Travel Miles"])

In [ ]:
(travel["New Weighted Existing Travel Miles"]/travel["Existing Travel"]).plot()

In [ ]:
#(travel["New Weighted Existing Travel"]/travel["Existing Travel"]).plot()

In [ ]:
(travel["Weighted Existing Travel"]/travel["Existing Travel"]).plot()

In [ ]:
travel["old_ratio"] = (travel["Weighted Existing Travel"]/travel["Existing Travel"])
travel.groupby("M Mode").plot(y=["old_ratio"])

In [ ]:
travel["new_ratio"] = (travel["New Weighted Existing Travel Miles"]/travel["Existing Travel"])
travel.groupby("M Mode").plot(y=["new_ratio"])

In [ ]:
travel.groupby("M Mode").plot(y=["New Weighted Existing Travel Miles"],legend=True)

In [ ]:
travel.groupby("M Mode")["New Weighted Existing Travel Miles"].plot(legend=True)

In [ ]:
travel.groupby("M Mode")["Weighted Existing Travel"].plot(legend=True)

In [ ]:
travel.groupby("M Mode")["Existing Travel"].plot(legend=True)

In [ ]:
travel.groupby("M Mode").plot(y=["Existing Travel","New Weighted Existing Travel Miles"],legend=True)

In [ ]:
travel.groupby("M Mode").plot(y=["Weighted Existing Travel","New Weighted Existing Travel Miles"],legend=True)

Basically for bicycling, the miles traveled is now less than the original volume, because it takes into account that each bicyclist may be counted multiple times. It should increase again with the adjacent factor?

## Step 2: Compare to the tool's current outputs
I won't be able to directly merge and compare these immediately because
1. My calculation isn't complete yet - still needs the "adjacent ways/intersections"
2. This table doesn't have the Project IDs - I would either need to ask to add them or try to merge it with the project list

For now, I'll just do a high level comparison of the overall average, min/max etc of the data (and how it looks on a graph)

In [ ]:
current = pd.read_csv('2024_01_18_reports_peter.csv')

In [ ]:
current.plot(y=["BMT (Total)","WMT (Total)"])

In [ ]:
current.plot(y=["BMT (Total)"])
travel[travel["M Mode"] == "bicycling"].plot(y=["New Weighted Existing Travel Miles"],legend=True)

In [ ]:
current["BMT (Total)"].describe()

In [ ]:
travel[travel["M Mode"] == "bicycling"]["New Weighted Existing Travel Miles"].describe()

In [ ]:
#current.sort_values(by="BMT (Total)").reset_index().plot(y=["BMT (Total)"])
#travel[travel["M Mode"] == "bicycling"].sort_values(by="New Weighted Existing Travel Miles").reset_index().plot(y=["New Weighted Existing Travel Miles"],legend=True)

In [ ]:
current.plot(y=["WMT (Total)"])
travel[travel["M Mode"] == "walking"].plot(y=["New Weighted Existing Travel Miles"],legend=True)